In [118]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../kaggle_prediction_library/') 
import preprocess
import feature_engineering
import submission
from sklearn.model_selection import train_test_split

# from hyperopt import tpe, fmin, Trials
# import hyperopt.hp as hp


In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import log_loss
from sklearn.feature_selection import chi2
from sklearn.metrics import r2_score
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss


### Read data

In [131]:
# For now we can use this, but we may need to recreate this once the teams have been announced
# The new version should only include feasible tourney matchups to make life easier
sub_df = pd.read_csv('SampleSubmission2024.csv', index_col=False)

In [132]:
regular_season_results = pd.read_csv('../data/MRegularSeasonDetailedResults.csv')
detailed_tourney_results = pd.read_csv('../data/MNCAATourneyDetailedResults.csv')
rankings = pd.read_csv('../data/MMasseyOrdinals.csv')
seeds = pd.read_csv('../data/MNCAATourneySeeds.csv')

# kp_rankings = pd.read_csv('../data/kenpom_pre_tourney_snapshot.csv')

regular_season_results_w = pd.read_csv('../data/WRegularSeasonDetailedResults.csv')
detailed_tourney_results_w = pd.read_csv('../data/WNCAATourneyDetailedResults.csv')

mteams = pd.read_csv('../data/MTeams.csv')
wteams = pd.read_csv('../data/WTeams.csv')

seeds_w = pd.read_csv('../data/WNCAATourneySeeds.csv')

# M538 = pd.read_csv('../data/M538.csv')
# W538 = pd.read_csv('../data/W538.csv')

seed_round = pd.read_csv("../data/MNCAATourneySeedRoundSlots.csv")
seeds = pd.read_csv("../data/MNCAATourneySeeds.csv")

first_round_odds_data = pd.read_csv('../data/first_round_odds_2024.csv')



In [122]:
# This is temporary, will remove once we are predicting 2024 instead of 2023
# detailed_tourney_results = detailed_tourney_results[detailed_tourney_results.Season < 2023].copy()
# detailed_tourney_results_w = detailed_tourney_results_w[detailed_tourney_results_w.Season < 2023].copy()

### Set Up Data

In [133]:
to_predict_mens, to_predict_womens, regular_season_games, regular_season_games_w = preprocess.full_setup(detailed_tourney_results, regular_season_results,
               detailed_tourney_results_w, regular_season_results_w,
               sub_df, mteams)

In [124]:
sub_df[ (sub_df.Team1 == 1194)
       & (sub_df.Team2 == 1321)]



,ID,Pred,Season,Team1,Team2
420,2024_1194_1321,0.5,2024,1194,1321


### Add Features

In [125]:
to_predict_womens = feature_engineering.TournamentSeed(tourney_seeds=seeds_w).add(to_predict_womens)
to_predict_womens = feature_engineering.Efficiency(games=regular_season_games_w, away_bonus=0).add(to_predict_womens)
to_predict_womens = feature_engineering.RoundNumber(seeds, seed_round).add(to_predict_womens)
to_predict_womens = feature_engineering.TeamNames(wteams).add(to_predict_womens)


# to_predict_womens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=W538).add(to_predict_womens)

In [126]:
#to_predict_womens.to_csv("../development_notebooks/to_predict_women.csv")
to_predict_womens.to_csv("to_predict_women.csv")

In [134]:
to_predict_mens = feature_engineering.TeamNames(mteams).add(to_predict_mens)
to_predict_mens = feature_engineering.FirstRoundOdds(first_round_odds_data).add(to_predict_mens)

In [135]:
to_predict_mens = feature_engineering.RoundNumber(seeds, seed_round).add(to_predict_mens)


In [136]:
to_predict_mens = feature_engineering.SeasonStats(regular_season_games).add(to_predict_mens)
# to_predict_mens = feature_engineering.FiveThirtyEight(fivethirtyeight_df=M538).add(to_predict_mens)
to_predict_mens = feature_engineering.PreSeasonAPRankings(rankings_df=rankings).add(to_predict_mens)
to_predict_mens = feature_engineering.TournamentSeed(tourney_seeds=seeds).add(to_predict_mens)
to_predict_mens = feature_engineering.Efficiency(games=regular_season_games, away_bonus=3).add(to_predict_mens)
to_predict_mens = feature_engineering.FinalRanking(rankings_df=rankings, system='WLK').add(to_predict_mens) # switched in 2024 because SAG dissapeared
# to_predict_mens = feature_engineering.Kenpom(kp_snapshot=kp_rankings).add(to_predict_mens)
# this one takes 3 minutes to run
# to_predict_mens = feature_engineering.TeamQuality(games=regular_season_games).add(to_predict_mens)


### Split Dataset

In [137]:
first_round_df = to_predict_mens[to_predict_mens.GameRound == 1].copy()
other_rounds_df = to_predict_mens[to_predict_mens.GameRound > 1].copy()


In [138]:
# first_round_df.to_csv("to_predict_mens_first_round.csv")
# other_rounds_df.to_csv("to_predict_mens_other_rounds.csv")
to_predict_mens.to_csv("to_predict_mens.csv")

In [24]:
other_rounds_df.groupby("type").count()

,ID,Pred,Season,Team1,Team2,Outcome,Gender,margin,t1_TeamName,t1_FirstD1Season,...,seed_diff,t1_adj_oe,t1_adj_de,t1_adj_margin,t2_adj_oe,t2_adj_de,t2_adj_margin,t1_final_rank,t2_final_rank,GameRound
type,,,,,,,,,,,,,,,,,,,,,
Historical,1178,0,1178,1178,1178,1178,1178,1178,1178,1178,...,1178,1178,1178,1178,1178,1178,1178,1178,1178,1178
Prediction,2238,2238,2238,2238,2238,0,2238,0,2238,2238,...,2238,2238,2238,2238,2238,2238,2238,2238,2238,2238
